3) The original model size is 10.7MB and the compressed model is 8.5MB. **It is 2MB smaller.**

In [1]:
import numpy as np
import tensorflow as tf
import os
import time
import PIL.Image as Image


In [2]:
path_to_model_folder = "saved_models/"
dataset_test = "dataset/Test"

In [3]:
# Convert the model (let's use the saved model from the previous exercise)
converter = tf.lite.TFLiteConverter.from_saved_model(path_to_model_folder + "model_1") # path to the SavedModel directory
tflite_model = converter.convert()


# Save the model
with open(path_to_model_folder + "model_1.tflite", 'wb') as f:
  f.write(tflite_model)

In [4]:
IMAGE_SHAPE = (224, 224)

def load_dataset():
    X_test = []
    y_test = []

    for classes in os.listdir(dataset_test):
        classes_path = os.path.join(dataset_test, classes)
        for image_name in os.listdir(classes_path):
            img = Image.open(os.path.join(classes_path, image_name)).resize(IMAGE_SHAPE)
            X_test.append(np.array(img, dtype=np.float32)/255.0)
            y_test.append(int(classes))
    return X_test, y_test

In [5]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']

X_test, y_test = load_dataset()

In [6]:
predicted_result = []
classifing_speed = []

for i in range(len(X_test)):
    start_time = time.time()
    interpreter.set_tensor(input_details[0]['index'], np.array(X_test[i][np.newaxis, ...]))
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    classifing_speed.append(time.time() - start_time)
    predicted_result.append(np.argmax(output_data[0], axis = -1))

In [7]:
average_time_per_image = sum(classifing_speed) / len(predicted_result)
print("Average time to classify an image is: ", average_time_per_image)

Average time to classify an image is:  0.03054928450748838


In [8]:
accuracy = 0.0

for i in range(len(predicted_result)):
    if (predicted_result[i] == y_test[i]):
        accuracy += 1

accuracy = accuracy / len(predicted_result)
print("Accuracy of TFlite model is: ", accuracy)

Accuracy of TFlite model is:  0.6275862068965518
